In [ ]:
import numpy as np
import pylab as plt
from skimage import io, filters, measure, feature, exposure, morphology, util
from scipy import ndimage
from scipy.ndimage import rotate
import os
import random
import inspect
%matplotlib inline

In [ ]:
toDraw = 'MobiusAlpha/'
ao_monkey = io.imread(toDraw + 'AO.png')
uv_monkey = io.imread(toDraw + 'UV.png')
normal_monkey = io.imread(toDraw + 'Normal.png')
original_monkey = io.imread(toDraw + 'Original.png')

fig=plt.figure(figsize=(20, 20))

fig.add_subplot(2, 2, 1)
plt.imshow(ao_monkey)
fig.add_subplot(2, 2, 2)
plt.imshow(uv_monkey)
fig.add_subplot(2, 2, 3)
plt.imshow(normal_monkey)
fig.add_subplot(2, 2, 4)
plt.imshow(original_monkey)
fig.tight_layout()

In [ ]:
edge_monkey = filters.sobel(ao_monkey[:,:,0])
edge_monkey = np.where(edge_monkey>0, 256, 0)
print(edge_monkey)
edge_monkey = ndimage.gaussian_filter(edge_monkey, sigma=0.6)
plt.figure(figsize=(20,20))
thinned_image = morphology.thin(np.where(edge_monkey<0.1, 0, 1))
thinned_image = np.where(thinned_image<0.1, 0, 1)
plt.imshow(thinned_image, cmap = plt.cm.Greys)

In [ ]:
new_edge_monkey =np.rot90(edge_monkey, k=-1)
contours = measure.find_contours(new_edge_monkey, 0.66, fully_connected="high")
simplified_contours = [measure.approximate_polygon(c, tolerance=0) for c in contours]
print(len(simplified_contours))
plt.figure(figsize=(6,6))
for n, contour in enumerate(simplified_contours):
    plt.plot(contour[:, 0], contour[:, 1], linewidth=1, color="#"+''.join([random.choice('0123456789ABCDEF') for j in range(6)]))

In [ ]:
histogram = exposure.histogram(original_monkey[:,:,0])
plt.figure()
plt.hist(histogram[0], histogram[1])

In [ ]:
original_monkey_shade = morphology.erosion(np.copy(original_monkey)[:,:,0] - edge_monkey)
shade_monkey = np.where(original_monkey_shade < 24, 256, 0)
print(np.where(original_monkey_shade < 24))
plt.figure(figsize=(20,20))
plt.imshow(original_monkey[:,:,1]-np.zeros_like(edge_monkey), cmap=plt.cm.Greys)

In [ ]:
vector_shade_monkey = np.rot90(shade_monkey, k=-1)
contours = measure.find_contours(vector_shade_monkey, 0.2, fully_connected="high")
simplified_contours = [measure.approximate_polygon(c, tolerance=0) for c in contours]
plt.figure(figsize=(6,6))
plt.axis("off")
for n, contour in enumerate(simplified_contours):
    plt.plot(contour[:, 0], contour[:, 1], linewidth=0.5, color="black")
plt.savefig("suzzane.svg")

In [ ]:
original_shade = morphology.erosion(np.copy(original_monkey)[:,:,0] - edge_monkey)
shaded_area = np.where(original_shade >= 60)
shade = np.where(original_shade < 60, 0, 256)
ao_image = np.where(ao_monkey[:,:,0]>0)
output = ao_monkey[:,:,0]-shade
output_coords = np.where(output>0)
output = np.where(output > 0, 256, 0)
mask = np.zeros_like(edge_monkey)
mask = util.random_noise(output, mode='gaussian', mean=0.01)
mask = np.where(output > 0, 256, 0)
output = output+mask-256
output = np.where(output > 0, 256, 0)
print(output_coords)
plt.imshow(output, cmap=plt.cm.Greys, vmax = 256, vmin = 0) 

In [ ]:
thin = morphology.thin(output)
thin = np.where(thin>0, 1, 0)
thin_coords = np.asarray(np.where(thin==1)).T
print(thin_coords)
plt.imshow(thin, cmap=plt.cm.Greys) 

In [ ]:
original_shade = morphology.erosion(np.copy(original_monkey)[:,:,0] - edge_monkey)
shaded_area = np.where(original_shade >= 24)
original_shade[::-8] = 256
shade = np.where(original_shade < 24, 0, 256)
shade[shaded_area] = 0
plt.imshow(shade, cmap=plt.cm.Greys, vmax = 256, vmin = 0) 

In [ ]:
#fd, hog_image = feature.hog(original_monkey[:,:,1], block_norm='L2-Hys', visualise=True)
#hog_image = exposure.rescale_intensity(hog_image)
object_coords = np.where(uv_monkey[:,:,1]!=0)
print(object_coords)
band_coords = np.where(uv_monkey[:,:,1]==99, 256, 0)
print(band_coords)
print(uv_monkey[:,:,1][166,474])
plt.imshow(band_coords)

In [ ]:
canvas = np.zeros_like(original_monkey[:,:,0])
canvas = util.random_noise(canvas, mode='gaussian', mean=0.01)
canvas = np.where(canvas > 0.1, 1, 0)
print(canvas)
plt.imshow(canvas, cmap=plt.cm.Greys)

In [ ]:
fig = plt.figure(figsize=(10,10))
u = fig.add_subplot(1,2,1)
u.set_title("U Map")
u_map = np.cos((uv_monkey[:,:,0]*2*np.pi)/200)
print(np.max(u_map), np.min(u_map))
plt.imshow(u_map)
v = fig.add_subplot(1,2,2)
v.set_title("V Map")
plt.imshow(uv_monkey[:,:,1])

In [ ]:
v_map = uv_monkey[:,:,1]
u_map = uv_monkey[:,:,0]
u_grad_axis1 = np.gradient(np.gradient(u_map, edge_order = 2, axis=1))
u_grad_axis1 = u_grad_axis1/np.max(u_grad_axis1)
u_grad_axis1 = np.sin(u_grad_axis1*2*np.pi)
fig = plt.figure(figsize=(40,40))
#fig.add_subplot(3,1,1)
#plt.imshow(u_grad_axis1)

u_grad_axis0 = np.gradient(u_map, edge_order = 2, axis=0)
u_grad_axis0 = u_grad_axis0/np.max(u_grad_axis0)
u_grad_axis0 = np.cos(u_grad_axis0*2*np.pi)
#fig.add_subplot(3,1,2)
#plt.imshow(u_grad_axis0)

fig.add_subplot(3,1,3)
final_once = u_grad_axis1+u_grad_axis0
final_once = np.where(final_once>0, 256, 0)
#plt.imshow(final_once)
print(u_grad_axis1.dtype)
v_grad_axis1 = np.gradient(v_map, edge_order=2, axis=1)
v_grad_axis1 = v_grad_axis1/np.max(v_grad_axis1)
fixed1 = np.zeros_like(u_grad_axis0)
fixed1[object_coords] = u_grad_axis0[object_coords]

fixed0 = np.zeros_like(u_grad_axis0)
fixed0[object_coords] = u_grad_axis0[object_coords]

skip = (slice(None, None, 5), slice(None, None, 5))
plt.quiver(np.flip(np.rot90(fixed1[skip]), axis=0), np.flip(np.rot90(fixed0[skip]), axis=0), units='width', pivot='mid')
plt.savefig("fig.svg")

In [ ]:
def clean_paths(paths):
    cleaned_paths = list()
    for path in paths:
        if len(cleaned_paths) == 0 or not np.array_equal(cleaned_paths[-1].vertices, path.vertices):
            if not np.allclose(path.vertices[0], path.vertices[1]):
                cleaned_paths.append(path)
    
    return cleaned_paths

def stitch_paths(paths):
    stitched_paths = list()
    for path in paths:
        if len(stitched_paths) == 0:
            stitched_paths.append(path.vertices)
        
        if np.allclose(stitched_paths[-1][-1], path.vertices[0]):
            joined = np.concatenate((stitched_paths[-1], np.array([path.vertices[1]])), axis=0)
            stitched_paths[-1] = joined
        else:
            stitched_paths.append(path.vertices)
            
    return stitched_paths

def makeContinuous(component):
    component = component/np.max(component)
    component = np.sin(component*2*np.pi)
    
    return component

In [ ]:
# grid = np.zeros_like(uv_monkey[:,:,0])
# grid[::2] = 1
# grid[:,::2] = 1
# plt.imshow(grid)

# h,w = grid.shape

# grid_points = np.array(np.where(grid == 1)).T
# grid_points[(grid_points[:,0] == 0), 0] += 1
# grid_points[(grid_points[:,1] == 0), 1] += 1
# grid_points[(grid_points[:,0] == h), 0] -= 1
# grid_points[(grid_points[:,1] == w), 1] -= 1

# print(np.where(grid_points[:,1] == h))
# print(grid_points[:,0], np.any(grid_points[:,0] == 0))

h, w = uv_monkey[:,:,0].shape
grid_points = np.mgrid[1:w-1:1, 1:h-1:1].reshape(2,-1).T

print(grid_points)

In [ ]:
uv_grad_U = np.gradient(makeContinuous(np.sqrt(uv_monkey[:,:,0].astype(np.float64))))
uv_grad_V = np.gradient(uv_monkey[:,:,1])

uv_grad_UX = uv_grad_U[0]
uv_grad_UY = uv_grad_U[1]

w, h = uv_grad_UX.shape
y, x = np.mgrid[0:w, 0:h]

fig, ax = plt.subplots()
im = ax.imshow(uv_monkey, cmap=plt.cm.Greys)
plt.axis('off')

stream = plt.streamplot(x, y,
                        uv_grad_UX, uv_grad_UY, 
                        density=25, linewidth=0.5, 
                        arrowsize=0.00000000001, integration_direction='both',
                        color='black', start_points=grid_points)

fig.canvas.draw()
plt.savefig('streamplot.svg')

plot = np.array(fig.canvas.renderer._renderer)
lines = stream.lines.get_paths()
print(len(lines))
cleaned_lines = clean_paths(lines)
print(len(cleaned_lines))
stitched_lines = stitch_paths(cleaned_lines)
print(len(stitched_lines))

In [ ]:
uv_grad_U = np.gradient(makeContinuous(uv_monkey[:,:,0].astype(np.float64)))
uv_grad_V = np.gradient(uv_monkey[:,:,1])

uv_grad_UX = uv_grad_U[0]
uv_grad_UY = uv_grad_U[1]

fig, ax = plt.subplots()
im = ax.imshow(uv_monkey, cmap=plt.cm.Greys)

plt.axis('off')

skip = (slice(None, None, 10), slice(None, None, 10))

quiver = plt.quiver(x[skip], y[skip], uv_grad_UX[skip], uv_grad_UY[skip], units='width', pivot='mid', headaxislength=0, scale=20, linewidth=0.1, headlength=0, rasterized = False)

fig.canvas.draw()
plt.savefig('quiverplot.svg')

lines = quiver.get_paths()
print(len(lines))
cleaned_lines = clean_paths(lines)
stitched_lines = stitch_paths(cleaned_lines)
print(len(stitched_lines))
print(stitched_lines[:5])

In [ ]:
print(stitched_lines)